In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Rastrigin Function
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Gradient of Rastrigin Function
def rastrigin_gradient(x):
    grad = []
    for xi in x:
        grad.append(2 * xi + 20 * np.pi * np.sin(2 * np.pi * xi))
    return np.array(grad)

# Lambda-CDM Optimizer
def lambda_cdm_optimizer(
    func, 
    dim, 
    bounds, 
    population_size=10, 
    max_iter=100,
    eta=0.01, 
    cluster_count=3
):
    # Initialization
    population = np.random.uniform(bounds[0], bounds[1], (population_size, dim))
    fitness = np.array([func(ind) for ind in population])
    best_solution = population[np.argmin(fitness)]
    best_fitness = np.min(fitness)
    alpha = np.random.uniform(0, 1)
    gamma = np.random.uniform(0, 1)

    # Convergence tracking
    convergence = [best_fitness]

    # Optimization loop
    for t in range(1, max_iter + 1):
        # Dark Energy Term
        lambda_t = 1 - t / max_iter  # Linear decay
        perturbations = lambda_t * np.random.normal(0, 1, (population_size, dim))
        population += perturbations

        # Attraction to Best Solution
        population += alpha * (best_solution - population)

        # Cold Dark Matter Term (Clustering)
        kmeans = KMeans(n_clusters=cluster_count, random_state=42)
        kmeans.fit(population)
        centroids = kmeans.cluster_centers_
        labels = kmeans.labels_

        for i, ind in enumerate(population):
            cluster_centroid = centroids[labels[i]]
            population[i] += gamma * (cluster_centroid - ind)

        # Ordinary Matter Term (Gradient Fine-tuning)
        for i in range(population_size):
            grad = rastrigin_gradient(population[i])
            population[i] -= eta * grad

        # Fitness Evaluation
        fitness = np.array([func(ind) for ind in population])
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_fitness:
            best_fitness = fitness[current_best_idx]
            best_solution = population[current_best_idx]

        # Record convergence
        convergence.append(best_fitness)

    return best_solution, best_fitness, convergence

# Particle Swarm Optimization
def pso_optimizer(func, dim, bounds, population_size=10, max_iter=100):
    # Initialization
    population = np.random.uniform(bounds[0], bounds[1], (population_size, dim))
    velocities = np.random.uniform(-1, 1, (population_size, dim))
    personal_best = population.copy()
    personal_best_fitness = np.array([func(ind) for ind in personal_best])
    global_best = personal_best[np.argmin(personal_best_fitness)]
    global_best_fitness = np.min(personal_best_fitness)
    w, c1, c2 = np.random.uniform(0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1)

    # Convergence tracking
    convergence = [global_best_fitness]

    # Optimization loop
    for t in range(max_iter):
        # Update velocities
        r1, r2 = np.random.rand(dim), np.random.rand(dim)
        velocities = (w * velocities 
                      + c1 * r1 * (personal_best - population) 
                      + c2 * r2 * (global_best - population))

        # Update positions
        population += velocities
        population = np.clip(population, bounds[0], bounds[1])

        # Evaluate fitness
        fitness = np.array([func(ind) for ind in population])
        for i in range(population_size):
            if fitness[i] < personal_best_fitness[i]:
                personal_best[i] = population[i]
                personal_best_fitness[i] = fitness[i]
        if np.min(personal_best_fitness) < global_best_fitness:
            global_best = personal_best[np.argmin(personal_best_fitness)]
            global_best_fitness = np.min(personal_best_fitness)

        # Record convergence
        convergence.append(global_best_fitness)

    return global_best, global_best_fitness, convergence

# Genetic Algorithm
def genetic_algorithm(func, dim, bounds, population_size=10, max_iter=100, mutation_rate=0.1):
    # Initialization
    population = np.random.uniform(bounds[0], bounds[1], (population_size, dim))
    fitness = np.array([func(ind) for ind in population])

    # Convergence tracking
    convergence = [np.min(fitness)]

    # Optimization loop
    for t in range(max_iter):
        # Selection
        parents = population[np.argsort(fitness)[:population_size // 2]]

        # Crossover
        offspring = []
        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                crossover_point = np.random.randint(1, dim)
                offspring1 = np.concatenate((parents[i][:crossover_point], parents[i + 1][crossover_point:]))
                offspring2 = np.concatenate((parents[i + 1][:crossover_point], parents[i][crossover_point:]))
                offspring += [offspring1, offspring2]

        # Mutation
        offspring = np.array(offspring)
        mutation = np.random.uniform(-1, 1, offspring.shape) * (np.random.rand(*offspring.shape) < mutation_rate)
        offspring += mutation
        offspring = np.clip(offspring, bounds[0], bounds[1])

        # New generation
        population = np.vstack((parents, offspring))
        fitness = np.array([func(ind) for ind in population])

        # Record convergence
        convergence.append(np.min(fitness))

    best_index = np.argmin(fitness)
    return population[best_index], np.min(fitness), convergence

# Parameters
dim = 2
bounds = [-5.12, 5.12]
population_size = 10
max_iter = 100

# Run all optimizers
_, _, lambda_cdm_convergence = lambda_cdm_optimizer(rastrigin, dim, bounds, population_size, max_iter)
_, _, pso_convergence = pso_optimizer(rastrigin, dim, bounds, population_size, max_iter)
_, _, ga_convergence = genetic_algorithm(rastrigin, dim, bounds, population_size, max_iter)

# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(lambda_cdm_convergence, label="Lambda-CDM Optimizer")
plt.plot(pso_convergence, label="Particle Swarm Optimization")
plt.plot(ga_convergence, label="Genetic Algorithm")
plt.title("Convergence of Optimizers on Rastrigin Function")
plt.xlabel("Iterations")
plt.ylabel("Best Fitness")
plt.legend()
plt.grid()
plt.show()
